## COSC 1210
### Example ball movement with cooldown

#### An example Class Ball

# From the previous notebook, it is changes so that...
- the balls start purple and go to blue, green, yellow, orange, and red on subsequent collisions
- they are deleted (with a list method) when they have had more than 5 collisions

In [ ]:
### ORIGINAL

import pygame
import random
import math

# --- Config ---
WIDTH, HEIGHT = 800, 600
BALL_RADIUS = 15
BG_COLOR = (20, 20, 30)
FPS = 60

# --- Ball Class ---
class Ball:
    def __init__(self):
        self.x = random.randint(BALL_RADIUS, WIDTH - BALL_RADIUS)
        self.y = random.randint(BALL_RADIUS, HEIGHT - BALL_RADIUS)
        self.vx = random.choice([-4, -3, 3, 4])
        self.vy = random.choice([-4, -3, 3, 4])
        self.color = (random.randint(50,255), random.randint(50,255), random.randint(50,255))

    def move(self):
        self.x += self.vx
        self.y += self.vy

        # Bounce off walls
        if self.x <= BALL_RADIUS or self.x >= WIDTH - BALL_RADIUS:
            self.vx *= -1
        if self.y <= BALL_RADIUS or self.y >= HEIGHT - BALL_RADIUS:
            self.vy *= -1

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, (self.x, self.y), BALL_RADIUS)

    def check_collision(self, other):
        dist = math.hypot(self.x - other.x, self.y - other.y)
        return dist < BALL_RADIUS * 2


# --- Main Game Loop ---
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Ball Collision Demo")
    clock = pygame.time.Clock()

    balls = []
    running = True

    while running:
        dt = clock.tick(FPS)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_b:
                    balls.append(Ball())

        # Move balls
        for ball in balls:
            ball.move()

        # Detect collisions
        for i in range(len(balls)):
            for j in range(i + 1, len(balls)):  # Avoid double checking
                if balls[i].check_collision(balls[j]):
                    # Change colors on collision
                    balls[i].color = (255, 0, 0)
                    balls[j].color = (255, 0, 0)

        # Draw any balls
        screen.fill(BG_COLOR)
        for ball in balls:
            ball.draw(screen)
        pygame.display.flip()

    pygame.quit()


if __name__ == "__main__":
    main()


In [1]:
### Updated

import pygame
import random
import math

# Config
WIDTH, HEIGHT = 800, 600
BALL_RADIUS = 15
BG_COLOR = (20, 20, 30)
FPS = 60
HIT_COOLDOWN_MS = 500  # ignore repeated collisions for this many ms

# Ball Class
class Ball:
    def __init__(self):
        self.x = random.randint(BALL_RADIUS, WIDTH - BALL_RADIUS)
        self.y = random.randint(BALL_RADIUS, HEIGHT - BALL_RADIUS)
        self.vx = random.choice([-4, -3, 3, 4])
        self.vy = random.choice([-4, -3, 3, 4])
        self.n_collisions = 0
        self.color = "purple"
        self.last_hit_ms = -HIT_COOLDOWN_MS  # ready immediately

    def move(self):
        self.x += self.vx
        self.y += self.vy

        # Bounce off walls only
        if self.x <= BALL_RADIUS or self.x >= WIDTH - BALL_RADIUS:
            self.vx *= -1
        if self.y <= BALL_RADIUS or self.y >= HEIGHT - BALL_RADIUS:
            self.vy *= -1

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), BALL_RADIUS)

    def check_collision(self, other):
        dist = math.hypot(self.x - other.x, self.y - other.y)
        return dist < BALL_RADIUS * 2

    def ready_for_hit(self, curr_ms):
        return (curr_ms - self.last_hit_ms) >= HIT_COOLDOWN_MS

    def register_hit(self, curr_ms):
        if self.n_collisions < 5:
            self.n_collisions += 1
            self.color = COLOR_MAP[self.n_collisions]
        self.last_hit_ms = curr_ms


# Game Loops
def main():
    pygame.init()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Ball Collision Demo - colors change!")
    clock = pygame.time.Clock()

    balls = []
    running = True
    # adding a 6th category is a little lazy but it works
    color_dict = {1:"blue", 2:"green", 3:"yellow", 4:"orange", 5:"red", 6:"red"}

    while running:
        dt = clock.tick(FPS)
        curr_ms = pygame.time.get_ticks()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_b:
                    balls.append(Ball())

        # Move balls
        for ball in balls:
            ball.move()

        # Check collisions
        # Introduce cooldown so colliding balls don't continue to collide as they pass through eachother
        to_remove = set()
        for i in range(len(balls)):
            for j in range(i + 1, len(balls)):
                if balls[i].check_collision(balls[j]):
                    # Only count if both are off cooldown
                    if balls[i].ready_for_hit(curr_ms) and balls[j].ready_for_hit(curr_ms):
                        if balls[i].n_collisions <= 5:
                            balls[i].n_collisions += 1
                            balls[i].color = color_dict[balls[i].n_collisions]
                            balls[i].last_hit_ms = curr_ms

                        if balls[j].n_collisions <= 5:
                            balls[j].n_collisions += 1
                            balls[j].color = color_dict[balls[j].n_collisions]
                            balls[j].last_hit_ms = curr_ms

        # remove balls that have more than 5 hits
        for ball in balls:
            if ball.ready_for_hit(curr_ms) and ball.n_collisions > 5:
                balls.remove(ball)

        # Draw everything
        screen.fill(BG_COLOR)
        for ball in balls:
            ball.draw(screen)
        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


KeyboardInterrupt: 